In [1]:
%load_ext autoreload
%autoreload 2
from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.molSize = (800, 800)
import numpy as np
import pandas as pd
from toolsets.search import string_search, quick_search_values, quick_search_sorted, search_feature
from tqdm import tqdm
import os
from toolsets.file_io import get_file_list
from rdkit import Chem
import toolsets.T_rex as trx
import toolsets.raw_data_scaffold as rds
import toolsets.feature_std_matching as fsm
import toolsets.denoising_related_functions as drf
from toolsets.std_list_prep import neutrilize_salt, neutrilize_salt_df, std_list_cleanup
from toolsets.std_list_prep import complete_adducts
import toolsets.spectra_operations as so
from toolsets.spectra_plotter import head_to_tail_plot, ms2_plot
import time

In [2]:
# master_dir ='/Volumes/scratch/Fanzhou/polyphenol_lib'
# mzml_dir = '/Volumes/scratch/Fanzhou/polyphenol_lib/mzml/pos'
master_dir ='/Volumes/Samsung_T5/polyphenol_lib/'
mzml_dir = '/Volumes/Samsung_T5/polyphenol_lib/pos_high_ce/mzml'
pl_dir = '/Volumes/Samsung_T5/polyphenol_lib/pos_high_ce/pl'
pos_adducts = ['[M]+','[M+H]+', '[M+Na]+', '[M+NH4]+']
neg_adducts = ['[M]-','[M-H]-','[M+Cl]-', '[M-H2O-H]-', ]

In [56]:
# std_list_pos = pd.read_csv(os.path.join(master_dir, 'std_list_pos.csv'))
# std_list_neg = pd.read_csv(os.path.join(master_dir, 'std_list_neg.csv'))
# std_list_nt = neutrilize_salt_df(std_list_pos, smile_column='smiles')
# std_list_cleaned =std_list_cleanup(std_list_nt)
# std_list_pos_adduct = complete_adducts(std_list_cleaned, adducts=pos_adducts)
# std_list_nt = neutrilize_salt_df(std_list_neg, smile_column='smiles')
# std_list_cleaned =std_list_cleanup(std_list_nt)
# std_list_neg_adduct = complete_adducts(std_list_cleaned, adducts=neg_adducts)
# std_list_pos_adduct.to_csv(os.path.join(master_dir, 'std_list_pos_adduct.csv'), index = False)
# std_list_neg_adduct.to_csv(os.path.join(master_dir, 'std_list_neg_adduct.csv'), index = False)

# above is still preparing std list

# C18 pos

In [2]:
master_dir ='/Volumes/Samsung_T5/polyphenol_lib/c18_pos'
mzml_dir = os.path.join(master_dir, 'mzml')
pl_dir = os.path.join(master_dir, 'pl')
eic_dir = os.path.join(master_dir, 'eic')
library_dir = os.path.join(master_dir, 'library')
for p in [ mzml_dir, pl_dir, eic_dir, library_dir]:
    if os.path.exists(p)==False:
        os.makedirs(p)

In [4]:
library_pos = pd.read_csv(os.path.join(library_dir, 'c18_pos.csv'))

In [17]:
library_pos

,precursor_mz,reference_mz,reference_adduct,reference_name,comment,reference_smiles,reference_mix,eic_center,reference_rt,eic_offset,...,ms1_intensity,n_scnas,ms1_scan_range,reci_snr,msms,ms2_scan_idx,reference_inchikey,rt_offset,msms_denoised,ei
0,717.145954,717.145011,[M+H]+,Theaflavin-3-gallate,Major,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(=O...,Polyphenols-pos-mix1,717.143131,2.958,0.011412,...,32374.0,152.0,"[862, 887, 1015]",7.116323e-04,45.59081661187863\t13.0\n55.02320638319634\t15...,1235.0,KMJPKUVSXFVQGZ-WQLSNUALSA-N,0.002737,55.02320638319634\t15.0\n65.0404250102491\t13....,91.18017
1,739.126771,739.126953,[M+Na]+,Theaflavin-3-gallate,Major,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(=O...,Polyphenols-pos-mix1,739.124880,2.958,0.011666,...,1756.0,136.0,"[879, 887, 1016]",1.326507e-04,161.02469594797503\t15.0\n323.07869375064274\t...,1225.0,KMJPKUVSXFVQGZ-WQLSNUALSA-N,0.002458,161.02469594797503\t15.0\n323.07869375064274\t...,87.42747
2,611.310553,611.311563,[M+H]+,Daurisoline,Major,COc1cc2c(cc1O)[C@@H](Cc1ccc(O)c(Oc3ccc(C[C@@H]...,Polyphenols-pos-mix1,611.308127,2.311,0.007430,...,379715.0,125.0,"[744, 751, 870]",0.000000e+00,42.0376165256764\t31.0\n42.48829065878977\t26....,918.0,BURJAQFYNVMZDV-FIRIVFDPSA-N,0.020040,42.0376165256764\t31.0\n44.04917359431113\t937...,94.20269
3,633.290548,633.293505,[M+Na]+,Daurisoline,Major,COc1cc2c(cc1O)[C@@H](Cc1ccc(O)c(Oc3ccc(C[C@@H]...,Polyphenols-pos-mix1,633.289846,2.311,0.009685,...,1829.0,67.0,"[746, 751, 814]",1.658694e-04,206.11880964599064\t62.0\n214.0962970614035\t9...,926.0,BURJAQFYNVMZDV-FIRIVFDPSA-N,0.018194,206.11880964599064\t62.0\n214.0962970614035\t9...,87.23404
4,647.194358,647.194638,[M+Na]+,Methyl-Hesperidin,Major,COc1ccc([C@@H]2CC(=O)c3c(O)cc(O[C@@H]4O[C@H](C...,Polyphenols-pos-mix1,647.190984,2.882,0.010245,...,62168.0,22.0,"[869, 876, 892]",0.000000e+00,127.03872098210357\t26.0\n131.057487897269\t26...,1189.0,GUMSHIGGVOJLBP-SLRPQMTOSA-N,0.001697,127.03872098210357\t26.0\n131.057487897269\t26...,88.54919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,801.278556,801.281170,[M+H]+,Jionoside A1,Major,COc1cc(/C=C/C(=O)O[C@@H]2[C@@H](CO[C@@H]3O[C@H...,Polyphenols-pos-mix65,801.277953,2.304,0.014102,...,7744.0,16.0,"[771, 781, 788]",1.706407e-02,83.05063679915555\t26.0\n85.02402982674762\t26...,910.0,UAPZTGRENZINFN-WEDRDYHSSA-N,0.060775,83.05063679915555\t26.0\n85.02402982674762\t26...,92.21790
800,818.303991,818.307717,[M+NH4]+,Jionoside A1,Major,COc1cc(/C=C/C(=O)O[C@@H]2[C@@H](CO[C@@H]3O[C@H...,Polyphenols-pos-mix65,818.303436,2.304,0.013581,...,73914.0,28.0,"[758, 781, 787]",3.223389e-05,43.0206464600506\t31.0\n46.99067889798821\t26....,906.0,UAPZTGRENZINFN-WEDRDYHSSA-N,0.061770,43.0206464600506\t31.0\n46.99067889798821\t26....,97.24867
801,314.138100,314.138684,[M+H]+,Laetanine,Major,COc1cc2c(cc1O)-c1c(OC)c(O)cc3c1[C@H](C2)NCC3,Polyphenols-pos-mix66,314.137800,1.959,0.005954,...,515904.0,56.0,"[706, 715, 763]",2.884924e-04,56.02730877600995\t26.0\n61.56200599374228\t26...,763.0,URQAEFLEDPPPFX-LBPRGKRZSA-N,0.000940,56.02730877600995\t26.0\n77.03282615828427\t26...,94.05358
802,741.221706,741.223655,[M+H]+,Apigenin-7-O-(2G-rhamnosyl)gentiobioside,Major,CC1OC(OC2C(Oc3cc(O)c4c(=O)cc(-c5ccc(O)cc5)oc4c...,Polyphenols-pos-mix66,741.218174,2.285,0.012164,...,278712.0,248.0,"[786, 808, 1035]",5.645339e-07,43.05750385208331\t31.0\n43.85399763284923\t26...,929.0,LXOPDILLGIDKLW-UHFFFAOYSA-N,0.006305,43.05750385208331\t31.0\n45.03275573250942\t31...,97.82888


In [19]:
drf.prep_formula(library_pos.iloc[1]['reference_smiles'], library_pos.iloc[1]['reference_adduct'])

'C36H28N2NaO16'

In [13]:
mass, intensity = so.break_spectra(library_pos.iloc[1]['msms_denoised'])

In [ ]:
import itertools

In [31]:
row = library_pos.iloc[1]

In [32]:
tt = so.truncate_msms(row['msms_denoised'], row['precursor_mz']-1.6)

In [40]:
ms1, ms2 = rds.read_mzml('Polyphenols-pos-mix1', mzml_dir)

In [41]:
search_feature(ms2, 739.126953)

,scan_idx,cycle,ms_level,precursor_mz,polarity,rt,peaks,isolation_window,ms1_pmz,ms1_rt,ms1_precursor_intensity,peak_purity,mz_offset,mix,base_name
339,1225,885,2,739.126083,+,2.9471,"[[161.02469594797503, 15.0], [323.078693750642...","(738.626083275606, 739.626083275606)",739.129422,2.943067,787.0,1.0,0.003339,Polyphenols-pos-mix1,Polyphenols-pos-mix1.mzML


In [39]:
library_pos

,precursor_mz,reference_mz,reference_adduct,reference_name,comment,reference_smiles,reference_mix,eic_center,reference_rt,eic_offset,...,ms1_intensity,n_scnas,ms1_scan_range,reci_snr,msms,ms2_scan_idx,reference_inchikey,rt_offset,msms_denoised,ei
0,717.145954,717.145011,[M+H]+,Theaflavin-3-gallate,Major,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(=O...,Polyphenols-pos-mix1,717.143131,2.958,0.011412,...,32374.0,152.0,"[862, 887, 1015]",7.116323e-04,45.59081661187863\t13.0\n55.02320638319634\t15...,1235.0,KMJPKUVSXFVQGZ-WQLSNUALSA-N,0.002737,55.02320638319634\t15.0\n65.0404250102491\t13....,91.18017
1,739.126771,739.126953,[M+Na]+,Theaflavin-3-gallate,Major,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(=O...,Polyphenols-pos-mix1,739.124880,2.958,0.011666,...,1756.0,136.0,"[879, 887, 1016]",1.326507e-04,161.02469594797503\t15.0\n323.07869375064274\t...,1225.0,KMJPKUVSXFVQGZ-WQLSNUALSA-N,0.002458,161.02469594797503\t15.0\n323.07869375064274\t...,87.42747
2,611.310553,611.311563,[M+H]+,Daurisoline,Major,COc1cc2c(cc1O)[C@@H](Cc1ccc(O)c(Oc3ccc(C[C@@H]...,Polyphenols-pos-mix1,611.308127,2.311,0.007430,...,379715.0,125.0,"[744, 751, 870]",0.000000e+00,42.0376165256764\t31.0\n42.48829065878977\t26....,918.0,BURJAQFYNVMZDV-FIRIVFDPSA-N,0.020040,42.0376165256764\t31.0\n44.04917359431113\t937...,94.20269
3,633.290548,633.293505,[M+Na]+,Daurisoline,Major,COc1cc2c(cc1O)[C@@H](Cc1ccc(O)c(Oc3ccc(C[C@@H]...,Polyphenols-pos-mix1,633.289846,2.311,0.009685,...,1829.0,67.0,"[746, 751, 814]",1.658694e-04,206.11880964599064\t62.0\n214.0962970614035\t9...,926.0,BURJAQFYNVMZDV-FIRIVFDPSA-N,0.018194,206.11880964599064\t62.0\n214.0962970614035\t9...,87.23404
4,647.194358,647.194638,[M+Na]+,Methyl-Hesperidin,Major,COc1ccc([C@@H]2CC(=O)c3c(O)cc(O[C@@H]4O[C@H](C...,Polyphenols-pos-mix1,647.190984,2.882,0.010245,...,62168.0,22.0,"[869, 876, 892]",0.000000e+00,127.03872098210357\t26.0\n131.057487897269\t26...,1189.0,GUMSHIGGVOJLBP-SLRPQMTOSA-N,0.001697,127.03872098210357\t26.0\n131.057487897269\t26...,88.54919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,801.278556,801.281170,[M+H]+,Jionoside A1,Major,COc1cc(/C=C/C(=O)O[C@@H]2[C@@H](CO[C@@H]3O[C@H...,Polyphenols-pos-mix65,801.277953,2.304,0.014102,...,7744.0,16.0,"[771, 781, 788]",1.706407e-02,83.05063679915555\t26.0\n85.02402982674762\t26...,910.0,UAPZTGRENZINFN-WEDRDYHSSA-N,0.060775,83.05063679915555\t26.0\n85.02402982674762\t26...,92.21790
800,818.303991,818.307717,[M+NH4]+,Jionoside A1,Major,COc1cc(/C=C/C(=O)O[C@@H]2[C@@H](CO[C@@H]3O[C@H...,Polyphenols-pos-mix65,818.303436,2.304,0.013581,...,73914.0,28.0,"[758, 781, 787]",3.223389e-05,43.0206464600506\t31.0\n46.99067889798821\t26....,906.0,UAPZTGRENZINFN-WEDRDYHSSA-N,0.061770,43.0206464600506\t31.0\n46.99067889798821\t26....,97.24867
801,314.138100,314.138684,[M+H]+,Laetanine,Major,COc1cc2c(cc1O)-c1c(OC)c(O)cc3c1[C@H](C2)NCC3,Polyphenols-pos-mix66,314.137800,1.959,0.005954,...,515904.0,56.0,"[706, 715, 763]",2.884924e-04,56.02730877600995\t26.0\n61.56200599374228\t26...,763.0,URQAEFLEDPPPFX-LBPRGKRZSA-N,0.000940,56.02730877600995\t26.0\n77.03282615828427\t26...,94.05358
802,741.221706,741.223655,[M+H]+,Apigenin-7-O-(2G-rhamnosyl)gentiobioside,Major,CC1OC(OC2C(Oc3cc(O)c4c(=O)cc(-c5ccc(O)cc5)oc4c...,Polyphenols-pos-mix66,741.218174,2.285,0.012164,...,278712.0,248.0,"[786, 808, 1035]",5.645339e-07,43.05750385208331\t31.0\n43.85399763284923\t26...,929.0,LXOPDILLGIDKLW-UHFFFAOYSA-N,0.006305,43.05750385208331\t31.0\n45.03275573250942\t31...,97.82888


In [4]:
std_list_pos = pd.read_csv(os.path.join(master_dir, 'std_list_pos_adduct.csv'))

In [24]:
for mix in tqdm(std_list_pos['mix'].unique()):
    ms1, ms2 = rds.read_mzml(mix, mzml_dir)
    mass_sorted, intensity_sorted, index_sorted, rt_list = trx.build_index(ms1)
    feature_temp = trx.get_features_beta(mass_sorted, intensity_sorted, index_sorted, rt_list)
    feature_temp = trx.map_ms2(feature_temp, ms2)[0]
    feature_temp.to_csv(os.path.join(pl_dir, mix+'.csv'), index = False)

100%|██████████| 66/66 [11:54<00:00, 10.83s/it]


In [91]:
matched_all = pd.DataFrame()
for mix in tqdm(std_list_pos['mix'].unique()):
    std_list_mix = string_search(std_list_pos, 'mix', mix)
    feature_temp = pd.read_csv(os.path.join(pl_dir, mix+'.csv'))
    matched_temp = fsm.feature_matching(feature_temp,std_list_mix, adducts= std_list_mix.columns[-4:])
    matched_all = pd.concat([matched_all, matched_temp], ignore_index=True)
    # break

100%|██████████| 66/66 [00:05<00:00, 11.29it/s]


In [148]:
from toolsets.denoising_related_functions import denoise_h

In [161]:
msms_denoised = [None]*len(matched_all)
ei = np.zeros(len(matched_all))
for index, row in tqdm(matched_all.iterrows(), total = len(matched_all)):
    msms_d, ei_t = denoise_h(row['msms'], row['reference_smiles'], row['reference_adduct'], row['precursor_mz'], reference_db_sorted=formula_db)
    msms_denoised[index]=msms_d
    ei[index]=ei_t

100%|██████████| 881/881 [22:45<00:00,  1.55s/it]  


In [162]:
matched_all['msms_denoised']=msms_denoised
matched_all['ei']=ei

In [169]:
matched_all_good = matched_all[matched_all['ei']>=np.mean(matched_all['ei'])-np.std(matched_all['ei'])]

In [170]:
matched_all_good.to_csv(os.path.join(library_dir, 'c18_pos.csv'), index = False)

In [173]:
from toolsets.file_io import export_library_msp

In [176]:
export_library_msp(matched_all_good, os.path.join(library_dir, 'c18_pos.txt'), typeofmsms='msms_denoised')

100%|██████████| 804/804 [00:01<00:00, 517.58it/s] 


# c18 neg

In [10]:
master_dir = '/Volumes/Samsung_T5/polyphenol_lib/c18_neg'
mzml_dir = os.path.join(master_dir, 'mzml')
pl_dir = os.path.join(master_dir, 'pl')
eic_dir = os.path.join(master_dir, 'eic')
library_dir = os.path.join(master_dir, 'library')
# for p in [mzml_dir, pl_dir, eic_dir, library_dir]:
#     if os.path.exists(p) == False:
#         os.makedirs(p)
std_list_neg = pd.read_csv(os.path.join(master_dir, 'std_list_neg_adduct.csv'))
# for mix in tqdm(std_list_neg['mix'].unique()):
#     ms1, ms2 = rds.read_mzml(mix, mzml_dir)
#     mass_sorted, intensity_sorted, index_sorted, rt_list = trx.build_index(ms1)
#     feature_temp = trx.get_features_beta(mass_sorted, intensity_sorted, index_sorted, rt_list)
#     feature_temp = trx.map_ms2(feature_temp, ms2)[0]
#     feature_temp.to_csv(os.path.join(pl_dir, mix + '.csv'), index=False)


In [33]:
matched_all

,precursor_mz,reference_mz,reference_adduct,reference_name,comment,reference_smiles,reference_mix,eic_center,reference_rt,eic_offset,...,rt_end,ms1_intensity,n_scnas,ms1_scan_range,reci_snr,msms,ms2_scan_idx,reference_inchikey,msms_denoised,ei
0,715.130849,715.130459,[M-H]-,Theaflavin-3-gallate,Major,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(=O...,Polyphenols-neg-mix1,715.129055,2.958,0.009828,...,4.781500,123935.0,646.0,"[941, 953, 1588]",0.000000e+00,57.03445407534312\t26.0\n75.8418774742191\t26....,1243.0,KMJPKUVSXFVQGZ-WQLSNUALSA-N,57.03445407534312\t26.0\n80.02407005636546\t26...,89.228655
1,751.104433,751.107137,[M+Cl]-,Theaflavin-3-gallate,Major,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(=O...,Polyphenols-neg-mix1,751.104631,2.958,0.012349,...,3.130467,3614.0,53.0,"[946, 953, 1000]",9.615490e-04,81.03152080488724\t13.0\n83.05389670711914\t13...,1250.0,KMJPKUVSXFVQGZ-WQLSNUALSA-N,81.03152080488724\t13.0\n83.05389670711914\t13...,92.904564
2,609.297118,609.297011,[M-H]-,Daurisoline,Major,COc1cc2c(cc1O)[C@@H](Cc1ccc(O)c(Oc3ccc(C[C@@H]...,Polyphenols-neg-mix1,609.295654,2.311,0.009379,...,3.415483,21015.0,348.0,"[753, 768, 1102]",1.140486e-04,93.03280900632357\t15.0\n105.03239014923282\t3...,981.0,BURJAQFYNVMZDV-FIRIVFDPSA-N,93.03280900632357\t15.0\n105.03239014923282\t3...,90.700935
3,300.999021,300.998991,[M-H]-,Ellagic acid,Major,O=c1oc2c(O)c(O)cc3c(=O)oc4c(O)c(O)cc1c4c23,Polyphenols-neg-mix1,300.998440,2.227,0.005296,...,2.663100,21984.0,191.0,"[680, 747, 872]",2.525933e-04,41.00577334616552\t6.0\n47.99751315016523\t5.0...,945.0,AFSDNFLWKVMVRB-UHFFFAOYSA-N,41.00577334616552\t6.0\n47.99751315016523\t5.0...,88.193689
4,623.200246,623.198145,[M-H]-,Methyl-Hesperidin,Major,COc1ccc([C@@H]2CC(=O)c3c(O)cc(O[C@@H]4O[C@H](C...,Polyphenols-neg-mix1,623.197191,2.882,0.008755,...,3.038850,293083.0,71.0,"[897, 940, 969]",9.666827e-06,58.150530955515976\t26.0\n59.01521625811365\t2...,1217.0,GUMSHIGGVOJLBP-SLRPQMTOSA-N,58.150530955515976\t26.0\n59.01521625811365\t2...,93.858340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,153.019559,153.019333,[M-H]-,"3,5-Dihydroxybenzoic acid",Major,O=C(O)c1cc(O)cc(O)c1,Polyphenols-neg-mix65,153.017891,1.102,0.004813,...,1.305500,37559.0,111.0,"[364, 400, 476]",2.109729e-07,40.88285820287027\t6.0\n41.0040834271509\t85.0...,436.0,UYEMGAFJOZZIFP-UHFFFAOYSA-N,41.0040834271509\t85.0\n45.00063721480446\t6.0...,88.461538
562,269.042572,269.045547,[M-H]-,Baicalein,Major,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,Polyphenols-neg-mix65,269.041345,NaN,0.004120,...,3.495633,829407.0,263.0,"[882, 1073, 1146]",0.000000e+00,41.00411242515802\t12.0\n45.001610449431084\t5...,1229.0,FXNFHKRTJBSTCS-UHFFFAOYSA-N,41.00411242515802\t12.0\n45.001610449431084\t5...,77.189409
563,191.034593,191.034983,[M-H]-,4-Methyldaphnetin,Major,Cc1cc(=O)oc2c(O)c(O)ccc12,Polyphenols-neg-mix65,191.033027,2.330,0.004502,...,3.156550,550885.0,279.0,"[766, 806, 1046]",7.993917e-06,40.88193380324141\t62.0\n41.004954875208036\t7...,1011.0,NWQBYMPNIJXFNQ-UHFFFAOYSA-N,41.004954875208036\t7960.0\n42.037621145958745...,93.022572
564,283.058526,283.061197,[M-H]-,Genkwanin,Major,COc1cc(O)c2c(=O)cc(-c3ccc(O)cc3)oc2c1,Polyphenols-neg-mix65,283.056085,4.108,0.004000,...,9.599183,1327856.0,1936.0,"[1331, 1346, 3268]",0.000000e+00,40.884626054426604\t31.0\n41.00585385802743\t6...,1796.0,JPMYFOBNRRGFNO-UHFFFAOYSA-N,41.00585385802743\t607.0\n43.045371854266214\t...,96.181335


In [12]:
ms1, ms2 = rds.read_mzml('Polyphenols-neg-mix1', mzml_dir)

In [13]:
mass_sorted, intensity_sorted, index_sorted, rt_list = trx.build_index(ms1)

In [21]:
ion_trace = trx.flash_eic(715.130459, mass_sorted, intensity_sorted, index_sorted, mass_error=0.1)

In [206]:
matched_all = pd.DataFrame()
for mix in tqdm(std_list_neg['mix'].unique()):
    std_list_mix = string_search(std_list_neg, 'mix', mix)
    feature_temp = pd.read_csv(os.path.join(pl_dir, mix + '.csv'))
    matched_temp = fsm.feature_matching(feature_temp, std_list_mix, adducts=std_list_mix.columns[-4:])
    matched_all = pd.concat([matched_all, matched_temp], ignore_index=True)
    # break


100%|██████████| 66/66 [00:03<00:00, 17.44it/s]


In [4]:
matched_all = pd.read_csv(os.path.join(pl_dir, 'matched_undenoised.csv'))

In [119]:
from rdkit import Chem

In [5]:
from toolsets.denoising_related_functions import denoise_h

msms_denoised = [None] * len(matched_all)
ei = np.zeros(len(matched_all))
for index, row in tqdm(matched_all.iterrows(), total=len(matched_all)):
    try:
        msms_d, ei_t = denoise_h(row['msms'], row['reference_smiles'], row['reference_adduct'], row['precursor_mz'],
                             reference_db_sorted=formula_db)
    except:
        msms_denoised[index] = row['msms']
        ei[index] = 100
        continue
    msms_denoised[index] = msms_d
    ei[index] = ei_t
matched_all['msms_denoised'] = msms_denoised
matched_all['ei'] = ei
matched_all_good = matched_all[matched_all['ei'] >= np.mean(matched_all['ei']) - np.std(matched_all['ei'])]
matched_all_good.to_csv(os.path.join(library_dir, 'c18_neg.csv'), index=False)


  0%|          | 0/497 [00:00<?, ?it/s]


KeyError: 'rt_offset'

In [7]:
matched_all_good['rt_offset']=np.abs(matched_all_good['rt_apex']-matched_all_good['reference_rt'])

In [45]:
all = pd.concat([matched_all_good, pos], ignore_index=True)

In [47]:
all['reference_inchikey'].unique()

array(['KMJPKUVSXFVQGZ-WQLSNUALSA-N', 'BURJAQFYNVMZDV-FIRIVFDPSA-N',
       'AFSDNFLWKVMVRB-UHFFFAOYSA-N', 'GUMSHIGGVOJLBP-SLRPQMTOSA-N',
       'GGQJXCQBBONZFX-IWVLMIASSA-N', 'DZAUWHJDUNRCTF-UHFFFAOYSA-N',
       'YQUVCSBJEUQKSH-UHFFFAOYSA-N', 'YWJXCIXBAKGUKZ-HJJNZUOJSA-N',
       'UCTWMZQNUQWSLP-UHFFFAOYSA-N', 'ZQSIJRDFPHDXIC-UHFFFAOYSA-N',
       'DFPMSGMNTNDNHN-ZPHOTFPESA-N', 'IPQVTOJGNYVQEO-KGFNBKMBSA-N',
       'AOJJSUZBOXZQNB-TZSSRYMLSA-N', 'QUQPHWDTPGMPEX-QJBIFVCTSA-N',
       'CUFLZUDASVUNOE-UHFFFAOYSA-N', 'DCYOADKBABEMIQ-OWMUPTOHSA-N',
       'CWBZAESOUBENAP-QVNVHUMTSA-N', 'WUADCCWRTIWANL-UHFFFAOYSA-N',
       'XLEYFDVVXLMULC-UHFFFAOYSA-N', 'CUCUKLJLRRAKFN-UHFFFAOYSA-N',
       'UQSRXQMIXSZGLA-UHFFFAOYSA-N', 'AEDDIBAIWPIIBD-ZJKJAXBQSA-N',
       'IBGBGRVKPALMCQ-UHFFFAOYSA-N', 'CDRPUGZCRXZLFL-OWOJBTEDSA-N',
       'AIONOLUJZLIMTK-AWEZNQCLSA-N', 'IQPNAANSBPBGFQ-UHFFFAOYSA-N',
       'DXDRHHKMWQZJHT-FPYGCLRLSA-N', 'SIXLXDIJGIWWFU-UHFFFAOYSA-N',
       'LNTHITQWFMADLM-UHFFFAOYSA-

In [50]:
548/640

0.85625

In [43]:
pos = pd.read_csv('/Volumes/Samsung_T5/polyphenol_lib/c18_pos/library/c18_pos.csv')

In [44]:
len(pos['reference_inchikey'].unique())/len(std_list_neg)

0.7890625

In [8]:
from toolsets.file_io import export_library_msp

export_library_msp(matched_all_good, os.path.join(library_dir, 'c18_neg.txt'), typeofmsms='msms_denoised')

100%|██████████| 497/497 [00:01<00:00, 384.92it/s] 


# Below is for neg

In [148]:
std_list_pos_adduct=pd.read_csv(os.path.join(master_dir, 'std_list_pos_adduct.csv'))
std_list_neg_adduct=pd.read_csv(os.path.join(master_dir, 'std_list_neg_adduct.csv'))

In [158]:
mzml_dir = '/Volumes/Samsung_T5/polyphenol_lib/neg/mzml'
pl_dir = '/Volumes/Samsung_T5/polyphenol_lib/neg/pl'
library_dir ='/Volumes/Samsung_T5/polyphenol_lib/neg/curated_library'
for p in [ library_dir]:
    if os.path.exists(p)==False:
        os.makedirs(p)

In [150]:
matched_all = pd.DataFrame()
for mix in std_list_neg_adduct['mix'].unique():
    try:
        matched_temp = pd.read_csv(os.path.join(pl_dir, mix+'_matched.csv'))
        matched_all = pd.concat([matched_all, matched_temp], ignore_index=True)
    except:
        pass

In [151]:
matched_all['rt_offset']=[abs(x-y) for x, y in zip(matched_all['reference_rt'], matched_all['rt_apex'])]

In [152]:
_tocheck = matched_all.query('rt_offset>0.1')

In [31]:
eic_dir ='/Volumes/Samsung_T5/polyphenol_lib/neg/EICs'

In [36]:
for index, row in tqdm(_tocheck.iterrows(), total = len(_tocheck)):
    if row['comment']=='Major':
        ms1, ms2 = read_mzml(row['reference_mix'], mzml_dir)
        rt_list, intensity_list = auto_EIC(row['reference_mz'],ms1)
        EIC(rt_list, intensity_list, vlines_location_1=[row['reference_rt']], vlines_location_2=[row['rt_apex']],
            savepath=os.path.join(eic_dir, row['reference_name']+'_'+row['reference_adduct']+'.png'), show=False)

100%|██████████| 27/27 [00:53<00:00,  1.98s/it]


In [153]:
from toolsets.denoising_related_functions import denoise_h

In [39]:
reference_db_sorted = pd.read_csv('db/formulaDB_sorted.csv')

In [154]:
msms_denoised = []
ei = []
for index, row in tqdm(matched_all.iterrows(), total = len(matched_all)):
    result_temp = denoise_h(row['msms'], row['reference_smiles'], row['reference_adduct'], reference_db_sorted, mass_error=0.02)
    msms_denoised.append(result_temp[0])
    ei.append(result_temp[1])

100%|██████████| 602/602 [01:58<00:00,  5.07it/s]


In [159]:
matched_all['ei']=ei
matched_all['msms_denoised']=msms_denoised

In [160]:
matched_good_quality = matched_all.query('ei>90 and rt_offset <=0.1')

In [161]:
matched_all.to_csv(os.path.join(library_dir, 'neg_matched_all.csv'), index = False)
matched_good_quality.to_csv(os.path.join(library_dir, 'neg_matched_good_quality.csv'), index = False)
export_library_msp(matched_good_quality, 'neg_matched_good_quality.txt', typeofmsms= 'msms_denoised',ifcollision_energy=False)
export_library_msp(matched_all, 'neg_matched_all.txt', typeofmsms= 'msms_denoised',ifcollision_energy=False)

In [273]:
matched_all=pd.read_csv(os.path.join(library_dir, 'neg_matched_all.csv'))
matched_good_quality = matched_all.query('ei>90')
rt_matched = []
for index, row in matched_good_quality.iterrows():
    if row['comment']=='Major' and row['rt_offset']>=0.1:
        rt_matched.append(False)
    else:
        rt_matched.append(True)
matched_good_quality['rt_matched']=rt_matched
matched_good_quality=matched_good_quality[matched_good_quality['rt_matched']==True]

In [276]:
matched_good_quality.to_csv(os.path.join(library_dir, 'neg_matched_good_quality.csv'), index = False)

In [ ]:
matched_all.to_csv(os.path.join(library_dir, 'neg_matched_all.csv'), index = False)
matched_good_quality.to_csv(os.path.join(library_dir, 'neg_matched_good_quality.csv'), index = False)
export_library_msp(matched_good_quality, 'neg_matched_good_quality.txt', typeofmsms= 'msms_denoised',ifcollision_energy=False)
export_library_msp(matched_all, 'neg_matched_all.txt', typeofmsms= 'msms_denoised',ifcollision_energy=False)

In [73]:
from toolsets.file_io import export_library_msp

In [79]:
export_library_msp(matched_good_quality, os.path.join(master_dir, 'neg_matched_good_quality.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)

100%|██████████| 524/524 [00:00<00:00, 1690.91it/s]


In [80]:
export_library_msp(matched_all, os.path.join(master_dir, 'neg_matched_all.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)

100%|██████████| 602/602 [00:00<00:00, 1698.21it/s]


# pos high ce

In [2]:
611.29-611.3115631

-0.021563099999980295

# neg high ce


In [278]:
std_list_pos_adduct=pd.read_csv(os.path.join(master_dir, 'std_list_pos_adduct.csv'))
std_list_neg_adduct=pd.read_csv(os.path.join(master_dir, 'std_list_neg_adduct.csv'))

In [279]:
mzml_dir = '/Volumes/Samsung_T5/polyphenol_lib/neg_high_ce/mzml'
pl_dir = '/Volumes/Samsung_T5/polyphenol_lib/neg_high_ce/pl'
eic_dir ='/Volumes/Samsung_T5/polyphenol_lib/neg_high_ce/EICs'
library_dir = '/Volumes/Samsung_T5/polyphenol_lib/neg_high_ce/curated_library'
for p in [eic_dir, library_dir]:
    if os.path.exists(p)==False:
        os.makedirs(p)

In [142]:
matched_all = pd.DataFrame()
for mix in std_list_neg_adduct['mix'].unique():
    try:
        matched_temp = pd.read_csv(os.path.join(pl_dir, mix+'_matched.csv'))
        matched_all = pd.concat([matched_all, matched_temp], ignore_index=True)
    except:
        pass

In [143]:
eic_dir ='/Volumes/Samsung_T5/polyphenol_lib/neg_high_ce/EICs'
library_dir = '/Volumes/Samsung_T5/polyphenol_lib/neg_high_ce/curated_library'
for p in [eic_dir, library_dir]:
    if os.path.exists(p)==False:
        os.makedirs(p)

In [144]:
matched_all['rt_offset']=[abs(x-y) for x, y in zip(matched_all['reference_rt'], matched_all['rt_apex'])]
_tocheck = matched_all.query('rt_offset>0.1')
for index, row in tqdm(_tocheck.iterrows(), total = len(_tocheck)):
    if row['comment']=='Major':
        ms1, ms2 = read_mzml(row['reference_mix'], mzml_dir)
        rt_list, intensity_list = auto_EIC(row['reference_mz'],ms1)
        EIC(rt_list, intensity_list, vlines_location_1=[row['reference_rt']], vlines_location_2=[row['rt_apex']],
            savepath=os.path.join(eic_dir, row['reference_name']+'_'+row['reference_adduct']+'.png'), show=False)

100%|██████████| 41/41 [00:58<00:00,  1.42s/it]


In [145]:
msms_denoised = []
ei = []
for index, row in tqdm(matched_all.iterrows(), total = len(matched_all)):
    result_temp = denoise_h(row['msms'], row['reference_smiles'], row['reference_adduct'], reference_db_sorted, mass_error=0.02)
    msms_denoised.append(result_temp[0])
    ei.append(result_temp[1])
matched_all['ei']=ei
matched_all['msms_denoised']=msms_denoised

100%|██████████| 599/599 [02:15<00:00,  4.43it/s]


In [146]:
# matched_good_quality = matched_all.query('ei>90 and rt_offset <=0.1')
# matched_all.to_csv(os.path.join(library_dir, 'neg_high_ce_matched_all.csv'), index = False)
# matched_good_quality.to_csv(os.path.join(library_dir, 'neg_high_ce_matched_good_quality.csv'), index = False)

In [280]:
matched_all=pd.read_csv(os.path.join(library_dir, 'neg_high_ce_matched_all.csv'))
matched_good_quality = matched_all.query('ei>90')
rt_matched = []
for index, row in matched_good_quality.iterrows():
    if row['comment']=='Major' and row['rt_offset']>=0.1:
        rt_matched.append(False)
    else:
        rt_matched.append(True)
matched_good_quality['rt_matched']=rt_matched
matched_good_quality=matched_good_quality[matched_good_quality['rt_matched']==True]

In [281]:
matched_good_quality.to_csv(os.path.join(library_dir, 'neg_high_ce_matched_good_quality.csv'), index = False)

In [284]:
export_library_msp(matched_good_quality, os.path.join(library_dir, 'neg_high_ce_matched_good_quality.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)

100%|██████████| 548/548 [00:00<00:00, 1252.46it/s]


In [147]:
export_library_msp(matched_good_quality, os.path.join(library_dir, 'neg_high_ce_matched_good_quality.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)
export_library_msp(matched_all, os.path.join(library_dir, 'neg_high_ce_matched_all.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)

100%|██████████| 599/599 [00:00<00:00, 1201.53it/s]


# pos

In [287]:
mzml_dir = '/Volumes/Samsung_T5/polyphenol_lib/pos/mzml'
pl_dir = '/Volumes/Samsung_T5/polyphenol_lib/pos/pl'
eic_dir ='/Volumes/Samsung_T5/polyphenol_lib/pos/EICs'
library_dir = '/Volumes/Samsung_T5/polyphenol_lib/pos/curated_library'
for p in [eic_dir, library_dir,mzml_dir,pl_dir]:
    if os.path.exists(p)==False:
        os.makedirs(p)

In [286]:
matched_all = pd.DataFrame()
for mix in std_list_pos_adduct['mix'].unique():
    try:
        matched_temp = pd.read_csv(os.path.join(pl_dir, mix+'.csv'))
        matched_all = pd.concat([matched_all, matched_temp], ignore_index=True)
    except:
        pass

In [120]:
eic_dir

'/Volumes/Samsung_T5/polyphenol_lib/neg_high_ce/EICs'

In [124]:
matched_all['rt_offset']=[abs(x-y) for x, y in zip(matched_all['reference_rt'], matched_all['rt_apex'])]
_tocheck = matched_all.query('rt_offset>0.1')
for index, row in tqdm(_tocheck.iterrows(), total = len(_tocheck)):
    if row['comment']=='Major':
        ms1, ms2 = read_mzml(row['reference_mix'], mzml_dir)
        rt_list, intensity_list = auto_EIC(row['reference_mz'],ms1)
        EIC(rt_list, intensity_list, vlines_location_1=[row['reference_rt']], vlines_location_2=[row['rt_apex']],
            savepath=os.path.join(eic_dir, row['reference_name']+'_'+row['reference_adduct']+'.png'), show=False)

100%|██████████| 48/48 [01:10<00:00,  1.47s/it]


In [127]:
msms_denoised = []
ei = []
for index, row in tqdm(matched_all.iterrows(), total = len(matched_all)):
    result_temp = denoise_h(row['msms'], row['reference_smiles'], row['reference_adduct'], reference_db_sorted, mass_error=0.02)
    msms_denoised.append(result_temp[0])
    ei.append(result_temp[1])
matched_all['ei']=ei
matched_all['msms_denoised']=msms_denoised

100%|██████████| 648/648 [02:10<00:00,  4.95it/s]


In [131]:
matched_good_quality

,precursor_mz,reference_mz,reference_adduct,reference_name,comment,reference_smiles,reference_mix,rt_apex,reference_rt,ms1_intensity,...,reci_snr,peak_purity,msms,ms1_scan_range,ms2_scan_idx_range_list,ms2_scan_idx,mix,reference_inchikey,ei,msms_denoised
0,717.143555,717.145011,[M+H]+,Theaflavin-3-gallate,Major,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(=O...,Polyphenols-pos-mix1,3.017704,2.958,2.613462e+05,...,0.002060,1.000000,50.558895111083984\t2925.380859375\n53.9635696...,"[651, 658, 705]",[1979],1979,Polyphenols-pos-mix1,KMJPKUVSXFVQGZ-WQLSNUALSA-N,90.38914,53.96356964111328\t3832.4111328125\n139.038894...
1,611.310428,611.311563,[M+H]+,Daurisoline,Major,COc1cc2c(cc1O)[C@@H](Cc1ccc(O)c(Oc3ccc(C[C@@H]...,Polyphenols-pos-mix1,2.350485,2.311,3.160802e+07,...,0.001026,0.996047,54.24769592285156\t31611.298828125\n58.0650901...,"[505, 512, 556]","[1537, 1597, 1552, 1612, 1567, 1525, 1582]",1537,Polyphenols-pos-mix1,BURJAQFYNVMZDV-FIRIVFDPSA-N,95.39166,54.24769592285156\t31611.298828125\n58.0650901...
2,625.211778,625.212696,[M+H]+,Methyl-Hesperidin,Major,COc1ccc([C@@H]2CC(=O)c3c(O)cc(O[C@@H]4O[C@H](C...,Polyphenols-pos-mix1,2.941834,2.882,9.698609e+06,...,0.000185,0.982593,54.1085205078125\t5332.501953125\n57.033763885...,"[635, 642, 679]","[1933, 1948, 1963, 1918]",1933,Polyphenols-pos-mix1,GUMSHIGGVOJLBP-SLRPQMTOSA-N,99.22022,54.1085205078125\t5332.501953125\n57.033763885...
3,642.238159,642.239244,[M+NH4]+,Methyl-Hesperidin,Major,COc1ccc([C@@H]2CC(=O)c3c(O)cc(O[C@@H]4O[C@H](C...,Polyphenols-pos-mix1,2.941481,2.882,4.183011e+06,...,0.000017,0.997564,51.89076614379883\t3361.51171875\n57.033565521...,"[635, 642, 674]","[1925, 1940]",1925,Polyphenols-pos-mix1,GUMSHIGGVOJLBP-SLRPQMTOSA-N,99.27629,57.033565521240234\t5839.18896484375\n69.03336...
6,329.086006,329.086708,[M+H]+,Bergenin,Major,COc1c(O)cc2c(c1O)[C@@H]1O[C@H](CO)[C@@H](O)[C@...,Polyphenols-pos-mix1,1.656435,1.658,4.049463e+06,...,0.000031,0.987607,50.02085494995117\t2723.94189453125\n55.017810...,"[342, 362, 398]","[1060, 1117, 1075, 1090, 1045, 1102]",1090,Polyphenols-pos-mix1,YWJXCIXBAKGUKZ-HJJNZUOJSA-N,98.67315,50.02085494995117\t2723.94189453125\n55.017810...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,818.306771,818.307717,[M+NH4]+,Jionoside A1,Minor,COc1cc(/C=C/C(=O)O[C@@H]2[C@@H](CO[C@@H]3O[C@H...,Polyphenols-pos-mix65,2.353838,2.304,3.326095e+06,...,0.000198,0.992216,57.03371810913086\t4749.8623046875\n58.6339340...,"[511, 517, 548]","[1550, 1565]",1550,Polyphenols-pos-mix65,UAPZTGRENZINFN-WEDRDYHSSA-N,99.59151,57.03371810913086\t4749.8623046875\n69.0333328...
643,314.138050,314.138684,[M+H]+,Laetanine,Major,COc1cc2c(cc1O)-c1c(OC)c(O)cc3c1[C@H](C2)NCC3,Polyphenols-pos-mix66,1.991807,1.959,7.110285e+07,...,0.004438,0.961760,56.80146789550781\t50045.47265625\n61.74120330...,"[433, 442, 510]","[1441, 1396, 1354, 1309, 1501, 1456, 1411, 153...",1324,Polyphenols-pos-mix66,URQAEFLEDPPPFX-LBPRGKRZSA-N,97.66315,70.10330963134766\t50888.48828125\n74.12525177...
644,314.138149,314.138684,[M+H]+,Laetanine,isomer,COc1cc2c(cc1O)-c1c(OC)c(O)cc3c1[C@H](C2)NCC3,Polyphenols-pos-mix66,1.889438,1.959,9.765207e+05,...,0.141130,0.984798,56.8175163269043\t2944.499755859375\n62.653366...,"[412, 420, 429]","[1261, 1276, 1246]",1261,Polyphenols-pos-mix66,URQAEFLEDPPPFX-LBPRGKRZSA-N,97.14487,191.08609008789062\t4849.83642578125\n209.0958...
646,741.222722,741.223655,[M+H]+,Apigenin-7-O-(2G-rhamnosyl)gentiobioside,Major,CC1OC(OC2C(Oc3cc(O)c4c(=O)cc(-c5ccc(O)cc5)oc4c...,Polyphenols-pos-mix66,2.332191,2.285,6.016567e+06,...,0.000252,0.991558,57.03361129760742\t22052.11328125\n64.34982299...,"[509, 516, 552]","[1540, 1555, 1570]",1540,Polyphenols-pos-mix66,LXOPDILLGIDKLW-UHFFFAOYSA-N,98.17302,57.03361129760742\t22052.11328125\n69.03345489...


In [130]:
matched_good_quality = matched_all.query('ei>90 and rt_offset <=0.1')
matched_all.to_csv(os.path.join(library_dir, 'pos_matched_all.csv'), index = False)
matched_good_quality.to_csv(os.path.join(library_dir, 'pos_matched_good_quality.csv'), index = False)

In [139]:
export_library_msp(matched_good_quality, os.path.join(library_dir, 'pos_matched_good_quality.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)
export_library_msp(matched_all, os.path.join(library_dir, 'pos_matched_all.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)

100%|██████████| 648/648 [00:00<00:00, 1637.45it/s]


In [288]:
matched_all=pd.read_csv(os.path.join(library_dir, 'pos_matched_all.csv'))
matched_good_quality = matched_all.query('ei>90')
rt_matched = []
for index, row in matched_good_quality.iterrows():
    if row['comment']=='Major' and row['rt_offset']>=0.1:
        rt_matched.append(False)
    else:
        rt_matched.append(True)
matched_good_quality['rt_matched']=rt_matched
matched_good_quality=matched_good_quality[matched_good_quality['rt_matched']==True]

In [290]:
matched_good_quality.to_csv(os.path.join(library_dir, 'pos_matched_good_quality.csv'), index = False)

In [291]:
export_library_msp(matched_good_quality, os.path.join(library_dir, 'pos_matched_good_quality.txt'), typeofmsms='msms_denoised', ifcollision_energy=False)

100%|██████████| 466/466 [00:00<00:00, 1622.38it/s]
